<a href="https://colab.research.google.com/github/Alirezarahhmati/Movie_Recommender_System/blob/Develop/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendatoin System

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alirezarahhmati","key":"73be1ed3ad085330a3f4400239645139"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download -d rounakbanik/the-movies-dataset

 94% 215M/228M [00:02<00:00, 103MB/s]
100% 228M/228M [00:02<00:00, 105MB/s]


In [4]:
! unzip /content/the-movies-dataset.zip -d /content/the_movies_dataset

Archive:  /content/the-movies-dataset.zip
  inflating: /content/the_movies_dataset/credits.csv  
  inflating: /content/the_movies_dataset/keywords.csv  
  inflating: /content/the_movies_dataset/links.csv  
  inflating: /content/the_movies_dataset/links_small.csv  
  inflating: /content/the_movies_dataset/movies_metadata.csv  
  inflating: /content/the_movies_dataset/ratings.csv  
  inflating: /content/the_movies_dataset/ratings_small.csv  


In [2]:
!pip install datasketch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.4 MB/s eta 0:00:00


## Overview Based

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
from datasketch import MinHashLSHForest, MinHash

In [102]:
df2 = pd.read_csv('/content/the_movies_dataset/movies_metadata.csv')

<ipython-input-102-7b9ba854191d>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/the_movies_dataset/movies_metadata.csv')


In [103]:
# Define a TF-IDF Vectorizer Object. Remove all English stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

In [104]:
# Replace NaN with an empty string
df2['overview'] = df2['overview'].fillna('')

In [105]:
# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['overview'])

In [106]:
# Generate MinHash signatures for each document
minhashes = []
for i in range(tfidf_matrix.shape[0]):
    vector = tfidf_matrix[i]
    doc_id = df2.index[i]
    minhash = MinHash(num_perm=128)
    for token in vector.nonzero()[1]:
        minhash.update(str(token).encode('utf-8'))
    minhashes.append((doc_id, minhash))

In [107]:
# Create LSH forest index
forest = MinHashLSHForest(num_perm=128)
for doc_id, minhash in minhashes:
    forest.add(doc_id, minhash)
forest.index()

In [108]:
# Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [117]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, top_k):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the MinHash signature for the query movie
    query_minhash = minhashes[idx][1]

    # Query the LSH forest to get approximate nearest neighbors
    nearest_neighbors = forest.query(query_minhash, top_k)

    # Get the movie indices of the nearest neighbors, excluding the input movie
    movie_indices = [int(nn) for nn in nearest_neighbors if nn != idx][:top_k]

    # Return the top k most similar movies
    return df2['title'].iloc[movie_indices]

In [124]:
get_recommendations('Shutter Island', 3)

5715                          Scanners
34779    Doctor Who: The Runaway Bride
Name: title, dtype: object

## Final Recommends

In [132]:
import random

In [156]:
def final_recommends(movies, result_number):
    res = []
    for i in range(len(movies)):
        recommendations = get_recommendations(movies[i], top_k=10)
        if len(recommendations) > 0:
          for j in range(9):
            res.append(recommendations.iloc[j])

    random.shuffle(res)

    return res[:result_number]

In [158]:
final_recommends(["Interstellar", "Shutter Island", "Inception"], 5)

['Turbo: A Power Rangers Movie',
 'High and Dizzy',
 'Brains Repaired',
 'Doctor Who: The Runaway Bride',
 'Star Trek: The Motion Picture']